## Pruning the network

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

In [35]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=4, validation_split=0.1)

Epoch 1/4
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3113 - accuracy: 0.9116 - val_loss: 0.1349 - val_accuracy: 0.9630
Epoch 2/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1349 - accuracy: 0.9619 - val_loss: 0.0971 - val_accuracy: 0.9752
Epoch 3/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0965 - accuracy: 0.9723 - val_loss: 0.0813 - val_accuracy: 0.9788
Epoch 4/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0776 - accuracy: 0.9774 - val_loss: 0.0776 - val_accuracy: 0.9798


In [36]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9763000011444092


In [37]:
import tensorflow_model_optimization as tfmot

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_4 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_4  (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [38]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=128, epochs=2, validation_split=0.1, callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 5s 10ms/step - loss: 0.0701 - accuracy: 0.9803 - val_loss: 0.0683 - val_accuracy: 0.9832
Epoch 2/2
422/422 [==============================] - 4s 9ms/step - loss: 0.0620 - accuracy: 0.9823 - val_loss: 0.0647 - val_accuracy: 0.9842


In [39]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9763000011444092
Pruned test accuracy: 0.9789999723434448
